# Homework 2: Naive Bayes Email Spam Filter

In this homework, we will learn how to implement the Naive Bayes classifier in order to create a simple email spam filter. This spam filter will be trained by test_emails, which will be given by a vector of tuples (emails, spam/nospam). For each tuple the first entry is a string ("email"), and the second entry is 0 or 1, depending whether the email contains spam words.

In [1]:
dictionary = ['hello', 'students', 'please', 'learn', 'for', 'the', 'exam', 'buy', 'drugs', 'today', 'sun', 'is', 'shining', 'in', 'nagoya', 'lets', 'sell', 'how', 'are', 'you', 'today?', 'do', 'your', 'homework', 'want', 'free', 'solutions?', 'hey', 'always', 'ask', 'questions', 'if','have', 'any', 'math', 'not', 'good', 'submit', 'pay'] 

test_emails=[
             ["hello students, please learn for the exam", 0],
             ["hello students, please buy drugs", 1],    
             ["hello, today the sun is shining in nagoya", 0],
             ["lets sell drugs in nagoya", 1],
             ["today learn drugs", 1],
             ["how are you today?", 0],
             ["hello students, please do your homework", 0],
             ["hello, do you want free homework solutions?", 1],
             ["hey students, please always ask questions if you have any", 0],
             ["math is not good", 1],
             ["math is good", 0],
             ["please submit your homework", 0],
             ["please buy questions", 1],
             ["please pay for the exam", 1]          
             ]

The feature space for our spam filter will be $\mathcal{X}=\{0,1\}^d$, where $d$ denotes the number of words in the dictionary. For a feature (email) $x \in \mathcal{X}$ the entry $x_i$ for $i=1,\dots,d$ is $1$ if the $i$-th word of the dictionary is contained in the email and $0$ otherwise.  

# **Exercise 1**
Implement a function which creates a feature vector out of an email and a function which creates a training set out of test emails. 

You would need to figure out whether a sentence contains a word, and there are functions in Python that could determine whether a string contains another string. You can consult documentation (and Google) to find out.

In [168]:
import numpy as np
def email_to_feature(dict0, email):
  feature = np.zeros(len(dict0))
  
  i = 0
  for word in dict0:
    if(word in email.split(" ")):
      feature[i] = 1
    i+=1

  return feature

def emails_to_test_samples(dict0, test_emails):
  result = []
  for email in test_emails:
    result.append([email_to_feature(dict0, email[0]), email[1]])
  return result

In [169]:
email_to_feature(dictionary, "pay")
emails_to_test_samples(dictionary, test_emails)

[[array([1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]), 0],
 [array([1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]), 1],
 [array([0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]), 0],
 [array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]), 1],
 [array([0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]), 1],
 [array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

 **Recall from Lecture 6:**

Given a training set  $\mathcal{T} = \left( (x^{(1)}, y^{(1)}) , \dots, (x^{(n)}, y^{(n)})   \right)$ we calculate for $i=1,\dots,d$ the following numbers
\begin{align*}
\phi_{i\mid y=1} &= \frac{1+\sum_{j=1}^n I(x^{(j)}_i = 1  \,\wedge \, y^{(j)}=1 ) }{2+\sum_{j=1}^n I(y^{(j)}=1)}\,,\\
	\phi_{i\mid y=0} &= \frac{1+\sum_{j=1}^n I(x^{(j)}_i = 1  \,\wedge \, y^{(j)}=0 )}{2+\sum_{j=1}^n I(y^{(j)}=0)}\,,\\
		\phi_{y=1} &= \frac{1+\sum_{j=1}^n I(y^{(j)} = 1)}{2+n} \,.
\end{align*}
Here $I$ denoted the indicator function. We used the laplace smoothing (thats why we have the $1+$ and $2+$) in order to make sure that we will not assume probability $0$ for unknown words.

Now assume we get a new feature (i.e. someone sends us an email) $x \in \{0,1\}^d$. Then we can calculate for each word $i=1,\dots,d$ the probabilities
\begin{align*}
P(x_i = 1 \mid y=1) &= \phi_{i\mid y=1}\,,\qquad &&P(x_i = 1 \mid y=0) = \phi_{i\mid y=0} \,,\\
P(x_i = 0 \mid y=1) &= 1- \phi_{i\mid y=1}\,,\qquad &&P(x_i = 0 \mid y=0) = 1-\phi_{i\mid y=0} \,. \\
\end{align*}

By the Naive Bayes assumption we have for $x \in \{0,1\}^d$
\begin{align*}
P(x \mid y)  = \prod_{i=1}^d P(x_j \mid y)\,.
\end{align*}
The probability of the new email being spam is then
\begin{align*}
P(y=1 \mid x) &= \frac{ P(x \mid y=1) P(y=1)}{P(x)}\\
&= \frac{\prod_{i=1}^d P(x_i \mid y = 1) \cdot \phi_{y=1}}{\prod_{i=1}^d P(x_i \mid y = 1) \cdot \phi_{y=1} + \prod_{i=1}^d P(x_i \mid y = 0) (1-\phi_{y=1})}\,.
\end{align*}


# **Exercise 2:** 
Use the above explanation of the Naive Bayes Spam filter and implement a function which gives the probability of an email being spam given the trainings email above. 


In [187]:
# You can implement the above algorithm in any way you want.
# One possible way:
# - Calculate the phis here
# - Write functions for the probability P(x|y) depending on these phis
# - Use this function in the function spam_percentage 
def laplace_smoothing(x, y):
  return (1 + x) / (2 + y)

def phi_probability_of_spam(training_set):
  # \phi_{y=1}

  spam_sum = 0

  for j in range(len(training_set)):
    x = training_set[j][0]
    y = training_set[j][1]

    if (y == 1):
      spam_sum += 1
  
  n = len(training_set)
  return laplace_smoothing(spam_sum, n)

  
def phi_likelihood(training_set, word_index_i, isSpamCondition):
  # \phi_{i|y=1}

  spam_contains_i_sum = 0
  for j in range(len(training_set)):
    x = training_set[j][0]
    y = training_set[j][1]

    if (x[word_index_i] == 1 and y == isSpamCondition):
      spam_contains_i_sum += 1

  
  spam_sum = 0
  for j in range(len(training_set)):
    x = training_set[j][0]
    y = training_set[j][1]

    if (y == 1):
      spam_sum += 1
  
  return laplace_smoothing(spam_contains_i_sum, 2+spam_sum)
  
def total_email_likelihood(training_set, dictionary, email, isSpamCondition):
  feature = email_to_feature(dictionary,email)
  product=1
  for i in range(len(feature)):
    if feature[i] == 0:
      product *= (1 - phi_likelihood(training_set, i, isSpamCondition))
    else:
      product *= phi_likelihood(training_set, i, isSpamCondition)
  return product

def how_probable_is_email_spam(training_set, dictionary, email):
  P_y1 = phi_probability_of_spam(training_set)
  spam_likelihood = total_email_likelihood(training_set, dictionary, email, 1)
  non_spam_likelihood = total_email_likelihood(training_set, dictionary, email, 0)
  P_x = P_y1 * spam_likelihood + (1-P_y1)*non_spam_likelihood

  return (P_y1 * spam_likelihood) / P_x


def spam_percentage(email, dict0):
  training_set = emails_to_test_samples(dict0,test_emails)
  print("ratio of spam", phi_probability_of_spam(training_set))
  return how_probable_is_email_spam(training_set, dict0, email)

Test your spam filter with the following email.


In [188]:
email="the sun is shining. buy drugs now"
print(spam_percentage(email, dictionary))

email="any homework submit sun nagoya"
print(spam_percentage(email, dictionary))

ratio of spam 0.5
0.9168255831010708
ratio of spam 0.5
0.14950166112956814



# **Exercise 3**
Extend your spamfilter by creating a dynamical dictionary. Instead of starting with a fixed dictionary, you should now create a dictionary out of a list of emails. 

Write a function `create_dictionary(emails)` which returns a dictionary created from a list of emails (Give as an array of arrays `[text, spam\nospam]`). Make sure that you do not include words more than once into the dictionary.
To implement this function you should look up the function `split()` for a string in Python. To take care of the symbols "." and "," you can use the `replace()` function of a string.

In [180]:
def create_dictionary(emails):
  word_list = []
  myDict = []
  for email in emails:
    email[0] = email[0].replace(",", "")
    email[0] = email[0].replace(".", "")
    word_list.extend(email[0].split(" "))
  
  for word in word_list:
    if word not in myDict:
      myDict.append(word)

  return myDict


In [181]:
create_dictionary(test_emails)

['hello',
 'students',
 'please',
 'learn',
 'for',
 'the',
 'exam',
 'buy',
 'drugs',
 'today',
 'sun',
 'is',
 'shining',
 'in',
 'nagoya',
 'lets',
 'sell',
 'how',
 'are',
 'you',
 'today?',
 'do',
 'your',
 'homework',
 'want',
 'free',
 'solutions?',
 'hey',
 'always',
 'ask',
 'questions',
 'if',
 'have',
 'any',
 'math',
 'not',
 'good',
 'submit',
 'pay']

In [186]:
myDict = create_dictionary(test_emails)
email="any homework submit sun nagoya"

print(email, spam_percentage(email,myDict))

ratio of spam 0.5
any homework submit sun nagoya 0.14950166112956814
